In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
def mape(y, y_pred):
    return np.mean(np.absolute(np.divide(np.subtract(y, y_pred), y)))

In [46]:
players = pd.read_csv("../data/player_data.csv")
train = pd.read_csv("../data/training_set.csv").drop("Game_ID", axis = 1)
train_y = train.pop("Rounded Viewers")
test = pd.read_csv("../data/test_set.csv").drop("Total_Viewers", axis = 1)
ids = test.pop("Game_ID")
game_data = pd.read_csv("../data/game_data.csv")

In [51]:
t1 = pd.get_dummies(train, columns = ["Away_Team", "Home_Team", "Country"])
t1["Season"] = le.fit_transform(t1["Season"])
t1["Game_Date"] = le.fit_transform(t1["Game_Date"])
model = RandomForestRegressor(n_estimators = 100)
#model.fit(train, train_y)
t1

,Season,Game_Date,Away_Team_ATL,Away_Team_BKN,Away_Team_BOS,Away_Team_CHA,Away_Team_CHI,Away_Team_CLE,Away_Team_DAL,Away_Team_DEN,...,Country_C90,Country_C91,Country_C92,Country_C93,Country_C94,Country_C95,Country_C96,Country_C97,Country_C98,Country_C99
0,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
